# Time Series Machine Learning Part 1 Assignment

In [1]:
import numpy as np
import pandas as pd
import plotly.express as px
from sklearn.ensemble import AdaBoostRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error

### Import the Netflix stock price data set (NFLX_data.csv).

In [41]:
netflix = pd.read_csv('https://tf-assets-prod.s3.amazonaws.com/tf-curric/data-science/Data%20Sets%20Time%20Series%20Analysis/Time%20Series%20-%20Day%205/NFLX_data.csv')
netflix.head()

,date,open,high,low,close,volume,Name
0,2013-02-08,25.9635,26.2800,25.7157,25.8528,25649820,NFLX
1,2013-02-11,25.5685,26.0071,24.9714,25.4128,29321782,NFLX
2,2013-02-12,25.8085,26.2228,25.1014,25.4214,34388802,NFLX
3,2013-02-13,25.8428,26.6285,25.6657,26.6098,40799094,NFLX
4,2013-02-14,26.7557,27.1214,26.3844,26.7714,31968685,NFLX


In [42]:
netflix['date'] = pd.to_datetime(netflix['date'])
netflix.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1259 entries, 0 to 1258
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   date    1259 non-null   datetime64[ns]
 1   open    1259 non-null   float64       
 2   high    1259 non-null   float64       
 3   low     1259 non-null   float64       
 4   close   1259 non-null   float64       
 5   volume  1259 non-null   int64         
 6   Name    1259 non-null   object        
dtypes: datetime64[ns](1), float64(4), int64(1), object(1)
memory usage: 69.0+ KB


In [43]:
netflix = netflix[['date', 'close']]
netflix.set_index('date', inplace=True)
netflix.head()

,close
date,
2013-02-08,25.8528
2013-02-11,25.4128
2013-02-12,25.4214
2013-02-13,26.6098
2013-02-14,26.7714


### Transform the data by shifting the series and creating features that will allow us to forecast the price 30 days into the future from 90 days of daily history.

In [44]:
history = 90
future = 30

shifts = [x + future for x in range(1, history + 1)]

In [45]:
for shift in shifts:
  netflix[f't-{shift}'] = netflix['close'].shift(shift)

In [64]:
netflix.columns

Index(['close', 't-31', 't-32', 't-33', 't-34', 't-35', 't-36', 't-37', 't-38',
       't-39', 't-40', 't-41', 't-42', 't-43', 't-44', 't-45', 't-46', 't-47',
       't-48', 't-49', 't-50', 't-51', 't-52', 't-53', 't-54', 't-55', 't-56',
       't-57', 't-58', 't-59', 't-60', 't-61', 't-62', 't-63', 't-64', 't-65',
       't-66', 't-67', 't-68', 't-69', 't-70', 't-71', 't-72', 't-73', 't-74',
       't-75', 't-76', 't-77', 't-78', 't-79', 't-80', 't-81', 't-82', 't-83',
       't-84', 't-85', 't-86', 't-87', 't-88', 't-89', 't-90', 't-91', 't-92',
       't-93', 't-94', 't-95', 't-96', 't-97', 't-98', 't-99', 't-100',
       't-101', 't-102', 't-103', 't-104', 't-105', 't-106', 't-107', 't-108',
       't-109', 't-110', 't-111', 't-112', 't-113', 't-114', 't-115', 't-116',
       't-117', 't-118', 't-119', 't-120'],
      dtype='object')

In [48]:
#dropna
netflix.dropna(inplace=True)
netflix.head()

,close,t-31,t-32,t-33,t-34,t-35,t-36,t-37,t-38,t-39,t-40,t-41,t-42,t-43,t-44,t-45,t-46,t-47,t-48,t-49,t-50,t-51,t-52,t-53,t-54,t-55,t-56,t-57,t-58,t-59,t-60,t-61,t-62,t-63,t-64,t-65,t-66,t-67,t-68,t-69,...,t-81,t-82,t-83,t-84,t-85,t-86,t-87,t-88,t-89,t-90,t-91,t-92,t-93,t-94,t-95,t-96,t-97,t-98,t-99,t-100,t-101,t-102,t-103,t-104,t-105,t-106,t-107,t-108,t-109,t-110,t-111,t-112,t-113,t-114,t-115,t-116,t-117,t-118,t-119,t-120
date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2013-08-01,35.5885,32.6900,32.7471,30.5700,30.7700,29.6628,30.6371,31.5614,31.4600,31.1057,31.9228,32.1871,31.7100,32.3214,31.8085,30.7628,30.5985,32.6771,32.3114,32.6514,33.8700,34.2214,34.1428,33.8614,34.7714,33.4243,32.7684,31.0991,30.9157,29.8014,29.4643,30.0985,30.4928,30.6414,30.4157,30.8671,30.7161,30.7928,30.5357,30.9600,...,23.2943,23.5228,23.8128,24.2485,25.2414,26.0614,27.0400,27.1771,27.2300,25.8271,25.9000,25.9985,26.1500,25.9100,26.5128,26.4071,26.9100,27.4800,26.0157,25.7785,26.3857,25.9371,26.1343,25.9614,25.8871,27.0528,26.8685,26.3314,26.2985,25.6171,25.6943,26.7357,26.7314,28.0643,27.0731,26.7714,26.6098,25.4214,25.4128,25.8528
2013-08-02,35.1685,33.1871,32.6900,32.7471,30.5700,30.7700,29.6628,30.6371,31.5614,31.4600,31.1057,31.9228,32.1871,31.7100,32.3214,31.8085,30.7628,30.5985,32.6771,32.3114,32.6514,33.8700,34.2214,34.1428,33.8614,34.7714,33.4243,32.7684,31.0991,30.9157,29.8014,29.4643,30.0985,30.4928,30.6414,30.4157,30.8671,30.7161,30.7928,30.5357,...,24.1943,23.2943,23.5228,23.8128,24.2485,25.2414,26.0614,27.0400,27.1771,27.2300,25.8271,25.9000,25.9985,26.1500,25.9100,26.5128,26.4071,26.9100,27.4800,26.0157,25.7785,26.3857,25.9371,26.1343,25.9614,25.8871,27.0528,26.8685,26.3314,26.2985,25.6171,25.6943,26.7357,26.7314,28.0643,27.0731,26.7714,26.6098,25.4214,25.4128
2013-08-05,36.2628,31.9314,33.1871,32.6900,32.7471,30.5700,30.7700,29.6628,30.6371,31.5614,31.4600,31.1057,31.9228,32.1871,31.7100,32.3214,31.8085,30.7628,30.5985,32.6771,32.3114,32.6514,33.8700,34.2214,34.1428,33.8614,34.7714,33.4243,32.7684,31.0991,30.9157,29.8014,29.4643,30.0985,30.4928,30.6414,30.4157,30.8671,30.7161,30.7928,...,23.7243,24.1943,23.2943,23.5228,23.8128,24.2485,25.2414,26.0614,27.0400,27.1771,27.2300,25.8271,25.9000,25.9985,26.1500,25.9100,26.5128,26.4071,26.9100,27.4800,26.0157,25.7785,26.3857,25.9371,26.1343,25.9614,25.8871,27.0528,26.8685,26.3314,26.2985,25.6171,25.6943,26.7357,26.7314,28.0643,27.0731,26.7714,26.6098,25.4214
2013-08-06,36.5571,30.9857,31.9314,33.1871,32.6900,32.7471,30.5700,30.7700,29.6628,30.6371,31.5614,31.4600,31.1057,31.9228,32.1871,31.7100,32.3214,31.8085,30.7628,30.5985,32.6771,32.3114,32.6514,33.8700,34.2214,34.1428,33.8614,34.7714,33.4243,32.7684,31.0991,30.9157,29.8014,29.4643,30.0985,30.4928,30.6414,30.4157,30.8671,30.7161,...,24.7157,23.7243,24.1943,23.2943,23.5228,23.8128,24.2485,25.2414,26.0614,27.0400,27.1771,27.2300,25.8271,25.9000,25.9985,26.1500,25.9100,26.5128,26.4071,26.9100,27.4800,26.0157,25.7785,26.3857,25.9371,26.1343,25.9614,25.8871,27.0528,26.8685,26.3314,26.2985,25.6171,25.6943,26.7357,26.7314,28.0643,27.0731,26.7714,26.6098
2013-08-07,35.6014,30.8000,30.9857,31.9314,33.1871,32.6900,32.7471,30.5700,30.7700,29.6628,30.6371,31.5614,31.4600,31.1057,31.9228,32.1871,31.7100,32.3214,31.8085,30.7628,30.5985,32.6771,32.3114,32.6514,33.8700,34.2214,34.1428,33.8614,34.7714,33.4243,32.7684,31.0991,30.9157,29.8014,29.4643,30.0985,30.4928,30.6414,30.4157,30.8671,...,24.7428,24.7157,23.7243,24.1943,23.2943,23.5228,23.8128,24.2485,25.2414,26.0614,27.0400,27.1771,27.2300,25.8271,25.9000,25.9985,26.1500,25.9100,26.5128,26.4071,26.9100,27.4800,26.0157,25.7785,26.3857,25.9371,26.1343,25.9614,25.8871,27.0528,26.8685,26.3314,26.2985,25.6171,25.6943,26.7357,26.7314,28.0643,27.0731,26.7714


### Split the data into a training set and a testing set. Make the test set size 20%.

In [49]:
#x = past shifts    y=future shifts
X = netflix.drop(['close'], axis=1)
y = netflix['close']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

In [50]:
X_train.tail()

,t-31,t-32,t-33,t-34,t-35,t-36,t-37,t-38,t-39,t-40,t-41,t-42,t-43,t-44,t-45,t-46,t-47,t-48,t-49,t-50,t-51,t-52,t-53,t-54,t-55,t-56,t-57,t-58,t-59,t-60,t-61,t-62,t-63,t-64,t-65,t-66,t-67,t-68,t-69,t-70,...,t-81,t-82,t-83,t-84,t-85,t-86,t-87,t-88,t-89,t-90,t-91,t-92,t-93,t-94,t-95,t-96,t-97,t-98,t-99,t-100,t-101,t-102,t-103,t-104,t-105,t-106,t-107,t-108,t-109,t-110,t-111,t-112,t-113,t-114,t-115,t-116,t-117,t-118,t-119,t-120
date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2017-03-08,137.39,138.60,138.41,133.26,132.89,133.70,129.18,130.50,129.89,130.95,131.07,131.81,129.41,127.49,123.80,125.33,125.89,128.35,125.59,125.58,126.50,125.12,125.45,124.22,125.00,123.44,123.78,122.83,122.88,123.24,125.39,124.57,119.16,120.81,117.22,117.00,117.51,116.93,117.41,117.69,...,124.34,124.58,122.03,122.14,122.34,123.30,124.87,126.57,126.47,126.97,126.51,127.33,127.50,123.35,121.87,118.79,99.80,101.47,100.23,99.50,100.59,103.33,104.82,105.07,106.28,102.34,102.63,98.55,96.67,97.48,97.07,94.56,95.94,95.83,94.88,98.25,98.06,99.48,97.34,97.01
2017-03-09,140.11,137.39,138.60,138.41,133.26,132.89,133.70,129.18,130.50,129.89,130.95,131.07,131.81,129.41,127.49,123.80,125.33,125.89,128.35,125.59,125.58,126.50,125.12,125.45,124.22,125.00,123.44,123.78,122.83,122.88,123.24,125.39,124.57,119.16,120.81,117.22,117.00,117.51,116.93,117.41,...,122.19,124.34,124.58,122.03,122.14,122.34,123.30,124.87,126.57,126.47,126.97,126.51,127.33,127.50,123.35,121.87,118.79,99.80,101.47,100.23,99.50,100.59,103.33,104.82,105.07,106.28,102.34,102.63,98.55,96.67,97.48,97.07,94.56,95.94,95.83,94.88,98.25,98.06,99.48,97.34
2017-03-10,139.52,140.11,137.39,138.60,138.41,133.26,132.89,133.70,129.18,130.50,129.89,130.95,131.07,131.81,129.41,127.49,123.80,125.33,125.89,128.35,125.59,125.58,126.50,125.12,125.45,124.22,125.00,123.44,123.78,122.83,122.88,123.24,125.39,124.57,119.16,120.81,117.22,117.00,117.51,116.93,...,115.42,122.19,124.34,124.58,122.03,122.14,122.34,123.30,124.87,126.57,126.47,126.97,126.51,127.33,127.50,123.35,121.87,118.79,99.80,101.47,100.23,99.50,100.59,103.33,104.82,105.07,106.28,102.34,102.63,98.55,96.67,97.48,97.07,94.56,95.94,95.83,94.88,98.25,98.06,99.48
2017-03-13,138.96,139.52,140.11,137.39,138.60,138.41,133.26,132.89,133.70,129.18,130.50,129.89,130.95,131.07,131.81,129.41,127.49,123.80,125.33,125.89,128.35,125.59,125.58,126.50,125.12,125.45,124.22,125.00,123.44,123.78,122.83,122.88,123.24,125.39,124.57,119.16,120.81,117.22,117.00,117.51,...,114.78,115.42,122.19,124.34,124.58,122.03,122.14,122.34,123.30,124.87,126.57,126.47,126.97,126.51,127.33,127.50,123.35,121.87,118.79,99.80,101.47,100.23,99.50,100.59,103.33,104.82,105.07,106.28,102.34,102.63,98.55,96.67,97.48,97.07,94.56,95.94,95.83,94.88,98.25,98.06
2017-03-14,142.45,138.96,139.52,140.11,137.39,138.60,138.41,133.26,132.89,133.70,129.18,130.50,129.89,130.95,131.07,131.81,129.41,127.49,123.80,125.33,125.89,128.35,125.59,125.58,126.50,125.12,125.45,124.22,125.00,123.44,123.78,122.83,122.88,123.24,125.39,124.57,119.16,120.81,117.22,117.00,...,113.38,114.78,115.42,122.19,124.34,124.58,122.03,122.14,122.34,123.30,124.87,126.57,126.47,126.97,126.51,127.33,127.50,123.35,121.87,118.79,99.80,101.47,100.23,99.50,100.59,103.33,104.82,105.07,106.28,102.34,102.63,98.55,96.67,97.48,97.07,94.56,95.94,95.83,94.88,98.25


In [51]:
X_test.head()

,t-31,t-32,t-33,t-34,t-35,t-36,t-37,t-38,t-39,t-40,t-41,t-42,t-43,t-44,t-45,t-46,t-47,t-48,t-49,t-50,t-51,t-52,t-53,t-54,t-55,t-56,t-57,t-58,t-59,t-60,t-61,t-62,t-63,t-64,t-65,t-66,t-67,t-68,t-69,t-70,...,t-81,t-82,t-83,t-84,t-85,t-86,t-87,t-88,t-89,t-90,t-91,t-92,t-93,t-94,t-95,t-96,t-97,t-98,t-99,t-100,t-101,t-102,t-103,t-104,t-105,t-106,t-107,t-108,t-109,t-110,t-111,t-112,t-113,t-114,t-115,t-116,t-117,t-118,t-119,t-120
date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2017-03-15,141.22,142.45,138.96,139.52,140.11,137.39,138.60,138.41,133.26,132.89,133.70,129.18,130.50,129.89,130.95,131.07,131.81,129.41,127.49,123.80,125.33,125.89,128.35,125.59,125.58,126.50,125.12,125.45,124.22,125.00,123.44,123.78,122.83,122.88,123.24,125.39,124.57,119.16,120.81,117.22,...,113.59,113.38,114.78,115.42,122.19,124.34,124.58,122.03,122.14,122.34,123.30,124.87,126.57,126.47,126.97,126.51,127.33,127.50,123.35,121.87,118.79,99.80,101.47,100.23,99.50,100.59,103.33,104.82,105.07,106.28,102.34,102.63,98.55,96.67,97.48,97.07,94.56,95.94,95.83,94.88
2017-03-16,140.71,141.22,142.45,138.96,139.52,140.11,137.39,138.60,138.41,133.26,132.89,133.70,129.18,130.50,129.89,130.95,131.07,131.81,129.41,127.49,123.80,125.33,125.89,128.35,125.59,125.58,126.50,125.12,125.45,124.22,125.00,123.44,123.78,122.83,122.88,123.24,125.39,124.57,119.16,120.81,...,115.19,113.59,113.38,114.78,115.42,122.19,124.34,124.58,122.03,122.14,122.34,123.30,124.87,126.57,126.47,126.97,126.51,127.33,127.50,123.35,121.87,118.79,99.80,101.47,100.23,99.50,100.59,103.33,104.82,105.07,106.28,102.34,102.63,98.55,96.67,97.48,97.07,94.56,95.94,95.83
2017-03-17,140.78,140.71,141.22,142.45,138.96,139.52,140.11,137.39,138.60,138.41,133.26,132.89,133.70,129.18,130.50,129.89,130.95,131.07,131.81,129.41,127.49,123.80,125.33,125.89,128.35,125.59,125.58,126.50,125.12,125.45,124.22,125.00,123.44,123.78,122.83,122.88,123.24,125.39,124.57,119.16,...,115.03,115.19,113.59,113.38,114.78,115.42,122.19,124.34,124.58,122.03,122.14,122.34,123.30,124.87,126.57,126.47,126.97,126.51,127.33,127.50,123.35,121.87,118.79,99.80,101.47,100.23,99.50,100.59,103.33,104.82,105.07,106.28,102.34,102.63,98.55,96.67,97.48,97.07,94.56,95.94
2017-03-20,139.20,140.78,140.71,141.22,142.45,138.96,139.52,140.11,137.39,138.60,138.41,133.26,132.89,133.70,129.18,130.50,129.89,130.95,131.07,131.81,129.41,127.49,123.80,125.33,125.89,128.35,125.59,125.58,126.50,125.12,125.45,124.22,125.00,123.44,123.78,122.83,122.88,123.24,125.39,124.57,...,115.21,115.03,115.19,113.59,113.38,114.78,115.42,122.19,124.34,124.58,122.03,122.14,122.34,123.30,124.87,126.57,126.47,126.97,126.51,127.33,127.50,123.35,121.87,118.79,99.80,101.47,100.23,99.50,100.59,103.33,104.82,105.07,106.28,102.34,102.63,98.55,96.67,97.48,97.07,94.56
2017-03-21,140.25,139.20,140.78,140.71,141.22,142.45,138.96,139.52,140.11,137.39,138.60,138.41,133.26,132.89,133.70,129.18,130.50,129.89,130.95,131.07,131.81,129.41,127.49,123.80,125.33,125.89,128.35,125.59,125.58,126.50,125.12,125.45,124.22,125.00,123.44,123.78,122.83,122.88,123.24,125.39,...,117.96,115.21,115.03,115.19,113.59,113.38,114.78,115.42,122.19,124.34,124.58,122.03,122.14,122.34,123.30,124.87,126.57,126.47,126.97,126.51,127.33,127.50,123.35,121.87,118.79,99.80,101.47,100.23,99.50,100.59,103.33,104.82,105.07,106.28,102.34,102.63,98.55,96.67,97.48,97.07


### Instantiate an AdaBoost model and fit it to the training set.

In [52]:
#ADA boost
model = AdaBoostRegressor()
model.fit(X_train, y_train)

AdaBoostRegressor(base_estimator=None, learning_rate=1.0, loss='linear',
                  n_estimators=50, random_state=None)

### Generate predictions for the test set.

In [54]:
predictions = model.predict(X_test)

### Evaluate the results using R-Squared, Mean Absolute Error, and Root Mean Squared Error metrics.

In [56]:
print('R-Squared', r2_score(y_test, predictions))
print('Mean Absolute Error:', mean_absolute_error(y_test, predictions))
print('Root Mean Squared Error:', np.sqrt(np.mean(mean_squared_error(y_test, predictions))))

R-Squared -4.648634191972618
Mean Absolute Error: 62.31783160165333
Root Mean Squared Error: 69.2697509278884


### Visually examine the results by creating a scatter plot where the x axis represents the observed results and the y axis represents the predictions.

In [57]:
def iscatter(df, x, y, color=None, size=None, title=''):
    fig = px.scatter(df, x=x, y=y, color=color, size=size, 
                     title=title, template='none')
    
    fig.update_traces(marker_line_color='black', 
                  marker_line_width=1)

    fig.show()

In [60]:
results = pd.DataFrame([list(y_test), predictions], index=['Observed', 'Predicted']).T
iscatter(results, 'Observed', 'Predicted', title='Observed vs. Predicted')